In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")
import opendatasets as od
import os
import cv2
from tqdm import *

In [2]:
od.download(dataset_id_or_url="https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces",data_dir="C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading 140k-real-and-fake-faces.zip to C:\Users\amith\Documents\Datasets\8 - Detecting DeepFakes with ResNet and CNN\140k-real-and-fake-faces


100%|██████████| 3.75G/3.75G [01:12<00:00, 55.1MB/s]


In [3]:
data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255.0)

In [4]:
train_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\train",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary",
)

val_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\valid",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary"
)

test_data = data_generator.flow_from_directory(
    directory = "C:\\Users\\amith\\Documents\\Datasets\\8 - Detecting DeepFakes with ResNet and CNN\\140k-real-and-fake-faces\\real_vs_fake\\real-vs-fake\\test",
    target_size = (64,64),
    color_mode = "grayscale",
    classes = ["fake","real"],
    class_mode = "binary"
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
model = tf.keras.models.Sequential(layers=[
    
    tf.keras.layers.Conv2D(filters=8,kernel_size=(3,3),activation="relu",input_shape=(64,64,1)),
    tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation="relu"),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation="relu"),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(units=64,activation="relu"),
    tf.keras.layers.Dense(units=1,activation="softmax")
])

In [6]:
model.compile(optimizer=tf.keras.optimizers.Adagrad(), loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])

In [7]:
model.fit(train_data,epochs=10,verbose=1,callbacks=tf.keras.callbacks.EarlyStopping(patience=2),validation_data=val_data,steps_per_epoch=100000/1000,validation_steps=20000/1000)

Epoch 1/10
100/100 [==============================] - 10s 91ms/step - loss: 0.6929 - accuracy: 0.4953 - val_loss: 0.6925 - val_accuracy: 0.4703
Epoch 2/10
100/100 [==============================] - 9s 86ms/step - loss: 0.6924 - accuracy: 0.4909 - val_loss: 0.6922 - val_accuracy: 0.4828
Epoch 3/10
100/100 [==============================] - 9s 90ms/step - loss: 0.6919 - accuracy: 0.5097 - val_loss: 0.6919 - val_accuracy: 0.4844
Epoch 4/10
100/100 [==============================] - 10s 98ms/step - loss: 0.6910 - accuracy: 0.5038 - val_loss: 0.6909 - val_accuracy: 0.4734
Epoch 5/10
100/100 [==============================] - 11s 113ms/step - loss: 0.6898 - accuracy: 0.4913 - val_loss: 0.6890 - val_accuracy: 0.5359
Epoch 6/10
100/100 [==============================] - 10s 97ms/step - loss: 0.6878 - accuracy: 0.4928 - val_loss: 0.6866 - val_accuracy: 0.4812
Epoch 7/10
100/100 [==============================] - 10s 99ms/step - loss: 0.6852 - accuracy: 0.5006 - val_loss: 0.6849 - val_accuracy: 

In [11]:
losses = model.evaluate(test_data,steps=100)

100/100 [==============================] - 7s 71ms/step - loss: 0.6710 - accuracy: 0.5050


In [14]:
losses

[0.6709726452827454, 0.5049999952316284]

In [16]:
pred_data = model.predict(test_data,steps=100)

100/100 [==============================] - 5s 50ms/step


In [28]:
len(test_data)

625

In [62]:
ctr = 1
p = []
q = []
for i in tqdm(range(len(test_data))):
    p += test_data[i][0].tolist()
    q += test_data[i][1].tolist()

100%|██████████| 625/625 [01:50<00:00,  5.65it/s]


In [ ]:
p = np.array(p)
q = np.array(q)

In [55]:
sum(p)

20000

In [43]:
test_data[0][0].shape

(32, 64, 64, 1)

In [44]:
test_data[0][1]

array([1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.],
      dtype=float32)

In [45]:
625 * 32

20000